In [1]:
from foreduce.fol import *
from foreduce.fol.random import *
from foreduce.transformer.config import *
from foreduce.fol.derivation import *

In [2]:
sig = RandomSignature(Config())
RandomClause(sig, 2)

p2_3(f1_2(X2), f1_2(c_2)) | ¬p3_0(f1_1(X2), X3, f1_1(c_1))

In [3]:
p = Predicate('p', 1)
X = Variable('X')
g = Function('g', 1)
f = Function('f', 1)
c = Constant('c')

In [4]:
literal1 = Literal(p(c), True)
literal2 = Literal(p(X), False)
literal3 = Literal(p(f(X)), True)
literal4 = Literal(eq(f(X), g(f(X))), True)
literal5 = Literal(p(f(g(f(c)))), False)

In [5]:
c1 = Clause(literal1)
c2 = Clause(literal2, literal3)
c3 = Clause(literal4)
c4 = Clause(literal5)

In [6]:
literal4 in c3.literals

True

In [7]:
literal1.substitute(c, g(c))

p(g(c))

In [18]:
Axiom(c3)

eq(f(X), g(f(X)))

In [ ]:
Axiom(c2)

In [17]:
sup = Superposition(Axiom(c3), Axiom(c2), c3.literals[0])
sup

p(X) | p(f(X)) | p(g(f(X))) | ¬p(X)

In [9]:
problem = Problem(c1, c2, c3, c4)
problem

p(c)
p(f(X)) | ¬p(X)
eq(f(X), g(f(X)))
¬p(f(g(f(c))))

In [10]:
from foreduce.transformer.config import Config
config = Config()
mapping = config.mapping(problem)

In [11]:
encoding = problem.encode(mapping)
len(encoding)

32

In [12]:
config.decode(encoding, mapping)

p(c)
p(f(X)) | ¬p(X)
eq(f(X), g(f(X)))
¬p(f(g(f(c))))

In [13]:
from foreduce.transformer.model import Transformer
transformer = Transformer(config)

In [14]:
config.vocab_size()

29

In [15]:
encoding += [0] * (config.seg_len - len(encoding))

In [16]:
import torch
transformer(torch.tensor([encoding])).shape

torch.Size([1, 128, 29])